<a href="https://colab.research.google.com/github/mariaferabelo/viagens-por-cargo-publico-2024/blob/main/Viagens_por_cargo_p%C3%BAblico_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
caminho_dados = "/content/drive/MyDrive/CursoPython/2024_Viagem.csv"
caminho_saida_tabela = "/content/drive/MyDrive/CursoPython/Output/tabela_2024.xlsx"
caminho_saida_grafico = "/content/drive/MyDrive/CursoPython/Output/grafico_2024.png"

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Lê os dados
df_viagens = pd.read_csv(caminho_dados, encoding="Windows-1252", sep=";", decimal = ",")

# Cria nova coluna de despesas
df_viagens['Despesas'] = df_viagens['Valor diárias'] + df_viagens['Valor passagens'] + df_viagens['Valor outros gastos']

# Ajusta valores nulos na coluna de cargo
df_viagens['Cargo'] = df_viagens['Cargo'].fillna('NÃO IDENTIFICADO')

# Converte colunas de datas
df_viagens['Período - Data de início'] = pd.to_datetime(df_viagens['Período - Data de início'], format='%d/%m/%Y')
df_viagens['Período - Data de fim'] = pd.to_datetime(df_viagens['Período - Data de fim'], format='%d/%m/%Y')

# Cria novas colunas de data
df_viagens['Mês da viagem'] = df_viagens['Período - Data de início'].dt.month_name()
df_viagens['Dias de viagem'] = (df_viagens['Período - Data de fim'] - df_viagens['Período - Data de início']).dt.days

In [ ]:
# Cria tabela consolidada
df_viagens_consolidado = (df_viagens
 .groupby('Cargo')
 .agg(despesa_media=('Despesas', 'mean'),
      duracao_media=('Dias de viagem', 'mean'),
      depesas_totais=('Despesas', 'sum'),
      destino_mais_frequente=('Destinos', pd.Series.mode),
      n_viagens=('Nome', 'count')
      )
 .reset_index()
 #.sort_values(by='despesas_totais', ascending=False)
 )

# Filtra tabela consolidada por cargos relevantes, que são os que representam mais de 1% das viagens
df_cargos = df_viagens['Cargo'].value_counts(normalize=True).reset_index()
cargos_relevantes = df_cargos.loc[df_cargos['proportion'] > 0.01, 'Cargo']
filtro = df_viagens_consolidado['Cargo'].isin(cargos_relevantes)

# Tabela final filtrada
df_final = df_viagens_consolidado[filtro].sort_values(by='n_viagens', ascending=False)

# Salvar tabela final
df_final.to_excel(caminho_saida_tabela, index=False)

In [ ]:
import matplotlib.pyplot as plt

# Cria a figura
fig, ax = plt.subplots(figsize=(16, 6))

# Plota o gráfico
ax.barh(df_final['Cargo'], df_final['n_viagens'], color='#058261')
ax.invert_yaxis()

# Ajusta o gráfico
ax.set_facecolor('black')
fig.suptitle('Viagens por cargo público 2024')
plt.figtext(0.75, 0.89, 'Fonte: Portal da Transparência')
plt.grid(color='gray', linestyle='--', linewidth=0.5)
plt.yticks(fontsize=8)
plt.xlabel('Número de viagens')
plt.ylabel('Cargo')

# Exibe o gráfico
plt.savefig(caminho_saida_grafico, bbox_inches='tight')